https://www.yeogi.com/<br>
오사카글자 넣어서 검색 <br>
9.24~9.27 검색 <br>
금액은 220,000만원 이하<br>
평점은 9.0 이상<br>
평가수는 5000명 이상입력한것만 출력하시오

In [12]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import random
from datetime import datetime
import pandas as pd

# 크롬 옵션 설정 - 셀레니움 접근 제한 : 보안
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 티 안 나게
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [13]:
# 브라우저 실행
browser = webdriver.Chrome(options=options) 
browser.maximize_window() # 창 최대화

# 1. 여기어때 접속하기
url = "https://www.yeogi.com/"
browser.get(url)
time.sleep(1)

# 2. 해외여행 클릭하기
elem = browser.find_element(By.XPATH,'//*[@id="LARGE_TAB_OVERSEAS_ACCOMMODATION"]').click()

In [14]:
time.sleep(1)
# 3. 오사카 글자 입력하기
elem = browser.find_element(By.CLASS_NAME,'css-59ixa7').click()
time.sleep(1)
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/main/section[1]/div/div/div[2]/form/div[1]/div[1]/div/label/div/div[2]/input')
elem.send_keys("오사카")
elem.send_keys(Keys.ENTER)
time.sleep(2)
# 오사카 선택하기
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/main/section[1]/div/div/div[2]/form/div[1]/div[2]/ul/li[1]/div[1]/div/span').click()

time.sleep(1)
# 4.날짜 선택하기
# 날짜 클릭
# elem = browser.find_element(By.XPATH,'//*[@id="__next"]/main/section[1]/div/div/div[2]/form/div[2]/div[1]/button/span').click()
# 9/24 선택하기위해서는 옆으로 넘겨야함
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/main/section[1]/div/div/div[2]/form/div[2]/div[2]/div/div[1]/div[1]/button[2]').click()
time.sleep(1)
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/main/section[1]/div/div/div[2]/form/div[2]/div[2]/div/div[1]/div[1]/button[2]').click()
time.sleep(1)
# 총 두번 넘김

# 9/24~9/27 선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/main/section[1]/div/div/div[2]/form/div[2]/div[2]/div/div[1]/div[2]/div[2]/div/div/ul[2]/li[25]/button/span').click()
time.sleep(1)
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/main/section[1]/div/div/div[2]/form/div[2]/div[2]/div/div[1]/div[2]/div[2]/div/div/ul[2]/li[28]/button/span').click()

# 검색 클릭
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/main/section[1]/div/div/div[2]/form/div[4]/button').click()

In [15]:
### 스크롤 내리기
# 현재 사이트 높이를 가져오는 자바스크립트
prev_height = browser.execute_script("return document.body.scrollHeight")

# 반복 실행
while True:
    # 현재 브라우저에서 0에서 현재 높이까지 스크롤 내리기
    # 자바 스크립트 실행 
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    # 변동된 현재 문서 높이를 가져오기
    curr_height = browser.execute_script("return document.body.scrollHeight")
    # 스크롤 높이가 변동이 없을 시 
    if curr_height == prev_height: break 
    prev_height = curr_height # 현재 높이를 다시 입력해서 스크롤 내리기 재실행
    
print("스크롤 내리기 종료")

스크롤 내리기 종료


In [16]:
#금액은 220,000만원 이하 평점은 8.0 이상 평가수는 5000명 이상입력한것만 출력하시오
soup = BeautifulSoup(browser.page_source,"lxml")

hotels = []
# 먼저 호텔 정보 모두 가져오기
data = soup.find("div",{"class":"css-1poun1e"})
hotel_list = data.find_all("div",{"class":"css-gvoll6"})
for hotel in hotel_list:
    #호텔이름
    title = hotel.find("h3",{"class":"gc-thumbnail-type-seller-card-title css-1gxx2ac"}).get_text().strip()
    print(title)
    # 평점
    rank = hotel.find("span",{"class":"css-9ml4lz"})
    rank = float(rank.get_text().strip()) # 타입 변경
    # 평가수
    rating = hotel.find("span",{"class":"css-oj6onp"}).get_text().strip().replace(",","")
    rating = int(rating[:-4])
    # 가격 
    price = hotel.find("span",{"class":"css-5r5920"}).get_text().strip().replace(",","")
    price = int(price)
    if rank < 8.0 or rating <= 5000 or price >= 220000:
        continue
    print(rank,rating)
    print(price)
    
    print('-'*50)   
    hotels.append({"호텔이름":title ,"평점":rank,"평가수":rating,"가격":price}) 

df = pd.DataFrame(hotels)
df.to_csv("hotels_list.csv", index=False, encoding="utf-8-sig")
    

더 싱굴라리 호텔 앤 스카이스파 앳 유니버설 스튜디오 재팬
하튼 호텔 신사이바시 나가호리도리
8.9 12367
132108
--------------------------------------------------
호텔 한큐 리스파이어 오사카
오사카 히노데 호텔 닛폰바시 - 심층수 온천
프리미어 호텔 캐빈 프레지던트 오사카
호텔 플라자 오사카
호텔 비스타 오사카 난바
호텔 몬토레 그라스미아 오사카
호텔 오리엔탈 익스프레스 오사카 신사이바시
8.9 5536
186505
--------------------------------------------------
난바 오리엔탈 호텔
베셀 인 남바
그리드스 프리미엄 호텔 오사카 남바
8.8 7197
162515
--------------------------------------------------
호텔 비나리오 우메다
8.0 5415
157364
--------------------------------------------------
베셀 인 신사이바시
8.9 6031
165135
--------------------------------------------------
퀸테사 호텔 오사카 신사이바시 코믹 & 북스
8.4 7220
166106
--------------------------------------------------
도톤보리 호텔
센타라 그랜드 호텔 오사카
칸데오 호텔 오사카 난바
